<a href="https://colab.research.google.com/github/mbc2009/Lmp_ML/blob/main/Model_trainning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Enviornment Initialization

In [1]:
%%capture
%%bash

# remove unnecessary
rm -rf *

# update pip
python -m pip install --upgrade pip

# install package
pip install opencv-python pillow
pip install segmentation_models_pytorch
pip install -q kaggle
pip install dropbox
pip install scikit-image

In [2]:
# basic import
import  os, sys, time, math, random, math, psutil
from    typing                  import  List, Tuple
from    dropbox                 import  Dropbox
from    tqdm                    import  tqdm
from    matplotlib              import  pyplot      as plt
import  numpy                                       as np
import  pandas                                      as pd
import  zipfile
import  warnings
import  shutil

from    skimage                 import  io
from    scipy                   import  interpolate
from    scipy.interpolate       import  RegularGridInterpolator
from    scipy.ndimage           import  generic_filter
from    PIL                     import  Image

import  torch
from    torch                   import  nn
from    torch.nn                import  functional  as F
import  torch.optim                                 as optim
import  torchvision.transforms.functional           as TF
from    torch.utils.data        import  Dataset, DataLoader, TensorDataset, random_split, Subset
from    torchvision             import  transforms, models
from    torchvision.transforms  import  *
import  kagglehub

In [3]:
# check hardware
print(f"CPU core #:\t{os.cpu_count()}")
print(f"CPU threads #:\t{psutil.cpu_count(logical=True)}")
print(f"Total memory:\t\t{psutil.virtual_memory().total / (1024**3):.2f} GB")
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"available GPU #:\t{gpu_count}")
    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i+1}:\t\t{gpu_name}")
else:
    print("No available GPU")

CPU core #:	2
CPU threads #:	2
Total memory:		12.67 GB
No available GPU


# 1. Configuration

In [4]:
class Confuration():
  # define paths to directory
  home_dir          = os.path.expanduser("~")
  working_dir       = os.path.join(os.getcwd(),'Lmp_ML')
  DataBase_dir      = os.path.join(working_dir,'DataBase')
  kaggle_config_dir = os.path.join(home_dir, ".kaggle")
  kaggle_json_dir   = os.path.join(working_dir, "kaggle.json")

conf = Confuration()

In [5]:
# import code from git hub
!git clone https://github.com/mbc2009/Lmp_ML

Cloning into 'Lmp_ML'...
remote: Enumerating objects: 506, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 506 (delta 20), reused 11 (delta 4), pack-reused 450 (from 3)
Receiving objects: 100% (506/506), 128.59 MiB | 20.38 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [6]:
# make directory
os.makedirs(conf.kaggle_config_dir, exist_ok=True)
os.makedirs(conf.DataBase_dir, exist_ok=True)

In [7]:
# download
original_path = kagglehub.dataset_download("mbc2009/heat-and-mass-transfer",force_download=True)
shutil.move(original_path, conf.DataBase_dir)

100%|██████████| 773M/773M [00:24<00:00, 32.4MB/s]

Extracting files...


'/content/Lmp_ML/DataBase/4'